## Modelo de datos de biblioteca de música

Este modelo de datos representa una biblioteca de música que tiene una gran colección de canciones y muestra sus canciones más descargadas casi en tiempo real.

Los patrones de acceso que este modelo de datos facilita son:

* Recuperación de un disco de canción, facilitada por una tabla llamada Songs.

* Recuperación de un registro de descarga específico o todos los registros de descarga de una canción, facilitada por una tabla llamada Songs.

* Recuperación de un registro de recuento mensual específico de descargas o todos los registros de recuento de descargas mensuales de una canción, facilitada por una tabla llamada Song.

* Recuperación de todos los registros (incluidos los registros de canciones, los registros de descargas y los registros de recuento de descargas mensuales) de una canción, facilitada por una tabla llamada Songs.

* Buscar la mayoría de las canciones descargadas, facilitado por el índice secundario global de la tabla Canciones llamado DownloadsByMonth.

In [1]:
import pandas as pd
import dynamodb
from dynamodb import DynamoTable
import json
from decimal import Decimal

In [2]:
dt = DynamoTable()

In [3]:
dt.create_table(
    table_name='Songs-c',
    partition_key='Id',
    partition_key_type='S',
    sort_key='Metadata',
    sort_key_type='S',
    provisioned=True,
    rcu=10,
    wcu=10
)

Table created successfully!


In [4]:
dt.select_table('Songs-c')
dt

- Table name: Songs-c            
- Table arn: arn:aws:dynamodb:us-east-1:853075046379:table/Songs-c            
- Table creation: 2023-03-06 15:55:50.548000-03:00            
- [{'AttributeName': 'Id', 'KeyType': 'HASH'}, {'AttributeName': 'Metadata', 'KeyType': 'RANGE'}]            
- [{'AttributeName': 'Id', 'AttributeType': 'S'}, {'AttributeName': 'Metadata', 'AttributeType': 'S'}]

In [5]:
# Itireate over json file and add items to table
with open('songs.json') as json_file:
    songs = json.load(json_file)
songs_form = json.loads(json.dumps(songs), parse_float=Decimal)

with dt.table.batch_writer() as batch:
    for song in songs_form:
        batch.put_item(Item=song)

In [10]:
df = dt.query_items(query="1", to_pandas=True, consumed_capacity=True)
df

Consumed Capacity: 0.5


,Title,Id,TotalDownloads,Metadata,Artist,DownloadTimestamp,DownloadMonth,TotalDownloadsInMonth
0,One,1,3,Details,Metallica,NaN,NaN,NaN
1,NaN,1,NaN,Dld-9349823681,NaN,2018-01-01T00:00:07,NaN,NaN
2,NaN,1,NaN,Dld-9349823682,NaN,2018-01-01T00:20:10,NaN,NaN
3,NaN,1,NaN,Month-01-2018,NaN,NaN,01-2018,3


In [12]:
dt__region__

NameError: name 'dt__region__' is not defined